# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
ls

Images/                               WeatherPy.ipynb
ImagesCity Latitude vs. Humidity.png  __pycache__/
VacationPy.ipynb                      api_keys.py
WeatherPy.csv


In [20]:

# This notebook requires that WeatherPy.ipynb is run first, to output data to the below csv file.

# Store the CSV created in WeatherPy into a dataframe.
city_df = pd.read_csv("WeatherPy.csv")
city_df.head()

,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Baykit,RU,1667600675,61.6700,96.3700,92,99,7.95,2.55
1,Ryōtsu-minato,JP,1667600428,38.0833,138.4333,81,59,51.67,9.40
2,Kaitangata,NZ,1667600675,-46.2817,169.8464,83,76,47.89,12.84
3,Bluff,NZ,1667600676,-46.6000,168.3333,95,63,50.22,24.76
4,Mataura,NZ,1667600676,-46.1927,168.8643,97,70,49.17,10.16


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# Determan max humidity level in the city data frame:
max_humidity = city['Humidity (%)'].max()
max_humidity

100

In [31]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)



In [49]:
gmaps.configure(api_key=g_key)
# Get the latitude and longitude.
locations = city_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_df["Max Temp (F)"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=[max(temp, 0) for temp in max_temp],
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
# Create new data frame with requested weather conditions:
city_narrow_down=city_df[(city_df['Max Temp (F)'] < 80) &
                             (city_df['Max Temp (F)'] > 65) &
                             (city_df['Wind Speed (mph)'] < 10) &
                             (city_df['Cloudiness (%)'] < 15) &
                             (city_df['Humidity (%)'] < 30)]
city_narrow_down=city_narrow_down.dropna() # drop any rows will null values
city_narrow_down.count()

City                5
Country             5
Date                5
Lat                 5
Lng                 5
Cloudiness (%)      5
Humidity (%)        5
Max Temp (F)        5
Wind Speed (mph)    5
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
city_narrow_down.head(6)

,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
109,Álamos,MX,1667600699,27.0167,-108.9333,0,22,78.93,5.55
225,Guerrero Negro,MX,1667600729,27.9769,-114.0611,2,29,73.40,9.98
453,Coyhaique,CL,1667600787,-45.5752,-72.0662,0,25,78.75,8.05
461,Araouane,ML,1667600788,18.9048,-3.5265,0,13,79.07,7.56
543,Kayes,ML,1667600804,14.0000,-11.0000,1,15,79.86,4.03


In [48]:
# Set up lists to hold response info:
name=[]         
city=[]         
country=[]      
lat=[]          
lng=[]          

# Loop through the list of cities and perform a request for data on each and add to corresponding lists:

# set counter to start value 1 for printouts of the current set count
count=1 

print('Beginning Data Retrieval')     
print('-----------------------------')
for i in range(len(city_narrow_down)) : 
   
    lati=str(city_narrow_down.iloc[i, 3])    
    lngi=str(city_narrow_down.iloc[i, 4])
    
    target_coordinates=lati + ', ' + lngi  
    target_search='Hotel'
    target_radius=5000
    target_type='lodging'

    # Set up a parameters dictionary:
    params={
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': g_key
    }

    # Base url:
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # Run a request using the above params dictionary:
    response=requests.get(base_url, params=params)
    #print(response.url)
    
    # Convert response to json:
    places_data=response.json()
    #print(json.dumps(places_data, indent=4, sort_keys=True))
    
    # Print user feedback: 
    print('Looking for Hotels near: ' + str(city_narrow_down.iloc[i, 0]) + ' | Record ' + str(count) + ' of Total ' + str(len(city_narrow_down)))
     
    # Add requested data to  result list
    try:                                             # ignore errors in response
        name.append(places_data['results'][0]['name'])
        city.append(city_narrow_down.iloc[i, 0])
        country.append(city_narrow_down.iloc[i, 1])
        lat.append(places_data['results'][0]['geometry']['location']['lat'])
        lng.append(places_data['results'][0]['geometry']['location']['lng'])
    except IndexError:
        print('No Hotel Near Location... Skipping...') # in case one of the responses comes back with error "IndexError"
                                                           # which could be no hotel near location 
  
    count=count+1
    
# Print Ending Log Statement:
print('-------------------------------')
print('Data Retrieval Complete')
print('-------------------------------')
    

Beginning Data Retrieval
-----------------------------
Looking for Hotels near: Álamos | Record 1 of Total 5
No Hotel Near Location... Skipping...
Looking for Hotels near: Guerrero Negro | Record 2 of Total 5
No Hotel Near Location... Skipping...
Looking for Hotels near: Coyhaique | Record 3 of Total 5
No Hotel Near Location... Skipping...
Looking for Hotels near: Araouane | Record 4 of Total 5
No Hotel Near Location... Skipping...
Looking for Hotels near: Kayes | Record 5 of Total 5
No Hotel Near Location... Skipping...
-------------------------------
Data Retrieval Complete
-------------------------------


In [50]:
# Create a data frame from api call list:
hotel_dict={
    'City': city,
    'Country': country,
    'Lat': lat,
    'Lng': lng,
    'Hotel Name': name
}
hotel_df=pd.DataFrame(hotel_dict)
hotel_df.count()

City          0
Country       0
Lat           0
Lng           0
Hotel Name    0
dtype: int64

In [51]:
hotel_df

,City,Country,Lat,Lng,Hotel Name


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
Name
{Hotel Name}

City
{City}
Country
{Country}
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Customize the size of the figure:
figure_layout={
    'width': '955px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig=gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable:
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map:
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [57]:

# Display figure# Create a combined map:
fig=gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))